In [7]:
!pip install pandas
!pip install matplotlib
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple seaborn
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple numpy

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
You are 

In [1]:
import pandas as pd
import numpy as np

# 正则化函数
def normalize_feature(df):
    return df.apply(lambda column: (column - column.mean()) / column.std())

# 载入csv
df = normalize_feature(pd.read_csv('data1.csv',
                                   names=['square', 'bedrooms', 'price']))

# ones是n行1列的数据框，表示x0恒为1
ones = pd.DataFrame({'ones': np.ones(len(df))})
# 根据列合并数据
df = pd.concat([ones, df], axis=1)
df.head()

,ones,square,bedrooms,price
0,1.0,0.130010,-0.223675,0.475747
1,1.0,-0.504190,-0.223675,-0.084074
2,1.0,0.502476,-0.223675,0.228626
3,1.0,-0.735723,-1.537767,-0.867025
4,1.0,1.257476,1.090417,1.595389


## 数据处理：获取 X 和 y

In [9]:
# [0:3]: 第0，1，2列
X_data = np.array(df[df.columns[0:3]])
# [-1]: 最后一列，reshape是为了把(47,0)变成形状(47,1)
y_data = np.array(df[df.columns[-1]]).reshape(len(df), 1)

print(X_data.shape, type(X_data))
print(y_data.shape, type(y_data))

for i in [0,1,2]:
    print(X_data.take(i,0))
    print(y_data.take(i))

((47, 3), <type 'numpy.ndarray'>)
((47, 1), <type 'numpy.ndarray'>)
[ 1.          0.13000987 -0.22367519]
0.4757468665785908
[ 1.         -0.50418984 -0.22367519]
-0.08407444031258184
[ 1.          0.50247636 -0.22367519]
0.2286257468223446


## 创建线性回归模型（数据流图）

In [4]:
import tensorflow as tf

alpha = 0.01 # 学习率 alpha
epoch = 500  # 训练全量数据集的轮数

# 用占位符定义输入数据的数据类型和形状
# 输入 X，类型float，形状[47, 3]
print(X_data.shape)
print(y_data.shape)
X = tf.placeholder(tf.float32, X_data.shape)
# 输出 y，类型float，形状[47, 1]
y = tf.placeholder(tf.float32, y_data.shape)

(47, 3)
(47, 1)


In [5]:
# 权重变量 W，形状(3,1)
#   X_data.shape[0]: 第0维的维度数47（47个样本）
#   X_data.shape[1]: 第1维的维度数3 （3个特征）
#   因此根据第2个参数，权重变量W的形状是 (3,1)
# 使用constant_initializer来初始化，初始化为全1
# tf.get_variable相比tf.variable可以从checkpoint中加载时可以快速找到这些变量的值
# 同时也支持变量的复用，单独的initializer的初始化
# W = tf.get_variable("weights", (X_data.shape[1], 1), initializer=tf.constant_initializer((1.0,1.0,1.0)))
W = tf.get_variable("weights", (X_data.shape[1], 1), initializer=tf.constant_initializer())

In [6]:
# 假设函数 h(x) = w0*x0+w1*x1+w2*x2, 其中x0恒为1
# 推理值 y_pred  形状[47,1]
y_pred = tf.matmul(X, W)

# 损失函数采用最小二乘法，y_pred - y 是形如[47, 1]的向量。
# tf.matmul(a,b,transpose_a=True) 表示：矩阵a的转置乘矩阵b，即 [1,47] X [47,1]
# tranpose_a = True表示对第一个参数做转置
# 损失函数操作 loss
loss_op = 1 / (2 * len(X_data)) * tf.matmul((y_pred - y), (y_pred - y), transpose_a=True)
# 随机梯度下降优化器 opt
opt = tf.train.GradientDescentOptimizer(learning_rate=alpha)
# 单轮训练操作 train_op
train_op = opt.minimize(loss_op)

## 创建会话（运行环境）

In [7]:
# for i in [0,1,2]:
#    print(X_data.take(i,0))
#    print(y_data.take(i))
# print(train_op)
# print(loss_op)

In [8]:
with tf.Session() as sess:
    # 初始化全局变量
    sess.run(tf.global_variables_initializer())
    # 开始训练模型，
    # 因为训练集较小，所以每轮都使用全量数据训练
    for e in range(1, epoch + 1):
        sess.run(train_op, feed_dict={X: X_data, y: y_data})
        if e % 10 == 0:
            loss, w = sess.run([loss_op, W], feed_dict={X: X_data, y: y_data})
            log_str = "Epoch %d \t Loss=%.4g \t Model: y = %.4gx1 + %.4gx2 + %.4g"
            print(log_str % (e, loss, w[1], w[2], w[0]))

Epoch 10 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 20 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 30 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 40 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 50 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 60 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 70 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 80 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 90 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 100 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 110 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 120 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 130 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 140 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 150 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 160 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 170 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 180 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 190 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 200 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 210 	 Loss=0 	 Model: y = 0x1 + 0x2 + 0
Epoch 220 	 Loss=0 	 Model: y = 0x1 + 0x2 +